In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

import cv2
import pandas as pd
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/transformation.csv')

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
#簡單的
x, y = df[df.columns[1:802]], df[df.columns[0]]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 1, stratify= y)
x_TT, x_val, y_TT, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state = 1, stratify= y_train)


#from sklearn.preprocessing import LabelEncoder 

In [8]:
#x_TT, x_test, y_TT , y_test, x_val, y_val

print(x_TT)
print("---------")
print(x_test)
print("---------")
print(x_val)
print("---------")
print(y_TT)
print("---------")
print(y_test)
print("---------")
print(y_val)


       0    1    2    3    4    5    6    7    8    9  ...  791  792  793  \
127  255  255  255  255  255  255  255  255  255  255  ...  255  255  255   
264  255  255  255  255  255  255  255  255  255  255  ...  255  255  255   
574  255  255  255  255  255  255  255  255  255  255  ...  255  255  255   
660  255  255  255  255  255  255  255  255  255  255  ...  255  255  255   
60   255  255  255  255  255  255  255  255  255  255  ...  255  255  255   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
222  255  255  255  255  255  255  255  255  255  255  ...  255  255  255   
685  255  255  255  255  255  255  255  255  255  255  ...  255  255  255   
252  255  255  255  255  255  255  255  255  255  255  ...  255  255  255   
427  255  255  255  255  255  255  255  255  255  255  ...  255  255  255   
205  255  255  255  255  255  255  255  255  255  255  ...  255  255  255   

     794  795  796  797  798  799  800  
127  255  255  255  255  255  255 

In [ ]:

#reshape


#標準化 min-max/另一個
#直接除255?

# mnist 的圖集黑色是255，白色是 0，而圖片讀進來是 黑色是0，白色是 255
# 所以在這裡除255後，也順便把黑白翻轉過來。
#data = 1 - data / 255.0 已經做了uint8

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator #利用現有的資料經過旋轉、翻轉、縮放…等方式增加更多的訓練資料
# train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
# test_datagen = ImageDataGenerator(rescale = 1./255)


# #要把資料做轉換的步驟則是用 .flow(X,y) 或是 .flow_from_directory(directory) 讀取資料集+批量生成器，產生每epoch訓練樣本
# training_set = train_datagen.flow_from_directory("./classify/dataset/training_set", target_size = (64, 64), batch_size = 5)
# test_set = test_datagen.flow_from_directory("./classify/dataset/test_set", target_size = (64, 64), batch_size = 5)


In [ ]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu')) #32 is no. of filters and kernel size is 5*5. ReLU is activation layer.s

model.add(MaxPooling2D(pool_size=(2,2))) #add Max pooling layer with kernel size 2*2 .

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=200)



#Evaluating model in keras
score = model.evaluate(x_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

model.fit_generator(training_set, epochs = 2, validation_data = test_set, verbose = 1)
score = model.evaluate_generator(test_set)


#model.save('EfficientNetB0-40-Stripped.h5', include_optimizer=False)如果你對這個模型滿意，想要保留之後使用的話，可以這樣設定儲存參數，那麼優化器的狀態不會被保存下來，可以節省不少體積，減少的體積量依使用優化器的不同而定，使用adam的話，這麼做是很有感的，如果你是之前中斷訓練，且有意後續載入繼續進行訓練的話，建議可以先不做。